# Bending Moment, Shear Force and Deflection Calculator
(For **statically determinate** beams only)

In [1]:
# DEPENDENCIES and DEFAULTS
import math #........................Math functionality
import numpy as np #.................Numpy for working with arrays

## 1.0 User data input

In [2]:
#DO NOT EDIT - Initialise containers (override below where req'd)
pointLoads = np.array([[]]) #Point Forces [location, xMag, yMag]
pointMoments = np.array([[]]) #Point Moments [location, mag] (clockwise is positive)
distributedLoads = np.array([[]]) #Uniformly distributed loads [xStart, xEnd, yMag]
linearLoads = np.array([[]]) #Distributed loads with linearly varying magnitude [xStart, xEnd, startMag, endMag]

In [3]:
#INPUT SPAN AND FORCE DATA BELOW
span = 3 #Span of beam
A = 0 #Distance to left support
B = 3 #Distance to right support

#Force data 
pointLoads = np.array([[0.5,0,-0.7],[1.0,0,-0.7],[2.0,0,-0.7],[2.5,0,-0.7]
                       ])

In [4]:
#Defaults and initialisations
delta = 0.005 
X = np.arange(0,span + delta,delta) #Range of x-coordinates
nPL = len(pointLoads[0]) #Test for point loads to consider
nPM = len(pointMoments[0]) #Test for uniformly distributed loads to consider
nUDL = len(distributedLoads[0]) #Test for uniformly distributed loads to consider
nLDL = len(linearLoads[0]) #Test for linearly varying magnitude loads to consider


#Initialise data containers
reactions = np.array([0.0,0,0]) #Reactions (Va, Ha, Vb) - Defined as array of floats
shearForce = np.empty([0,len(X)]) #Shear forces at each data point
bendingMoment = np.empty([0,len(X)]) #Bending moment at each data point

<hr>

## 2.0 Reaction Calculation

**2.1 Define a function to calculate reactions due to <u>point loads</u>**

In [5]:
def reactions_PL(n):       
    xp = pointLoads[n,0] #Location of point load
    fx = pointLoads[n,1] #Point load horizontal component magnitude 
    fy = pointLoads[n,2] #Point load vertical component magnitude 
    
    la_p = A-xp  #Lever arm of point load about point A
    mp = fy*la_p #Moment generated by point load about A - closkwise moments are positive
    la_vb = B-A  #Lever arm of vertical reaction at B about point A

    Vb = mp/la_vb #Vertical reaction at B
    Va = -fy-Vb   #Vertical reaction at A    
    Ha = -fx      #Horizontal reaction at A
    
    return Va, Vb, Ha    

**2.2 Define a function to calculate reactions due to <u>point moments</u>**

In [6]:
def reactions_PM(n):       
    xm = pointMoments[n,0] #Location of point moment
    m = pointMoments[n,1]  #Point moment magnitude    
   
    la_vb = B-A #Lever arm of vertical reaction at B about point A

    Vb = m/la_vb #Vertical reaction at B
    Va = -Vb     #Vertical reaction at A    
    
    return Va, Vb

**2.3 Define a function to calculate reactions due to <u>uniformly distributed loads</u> (UDL)**

In [7]:
def reactions_UDL(n): 
    xStart = distributedLoads[n,0] #Location of start of UDL
    xEnd = distributedLoads[n,1]   #Location of end of UDL
    fy = distributedLoads[n,2]     #Magnitude of UDL
    
    fy_Res = fy*(xEnd-xStart)          #Magnitude of resultant 
    x_Res = xStart + 0.5*(xEnd-xStart) #Location of resultant
    
    #SAME AS POINT LOAD FUNCTION FROM HERE 
    la_p = A-x_Res   #Lever arm of resultant load about point A
    mp = fy_Res*la_p #Moment generated by resultant load about A - closkwise moments are positive
    la_vb = B-A      #Lever arm of vertical reaction at B about point A
    
    Vb = mp/la_vb   #Vertical reaction at B
    Va = -fy_Res-Vb #Vertical reaction at A    
    
    return Va, Vb

**2.4 Define a function to calculate reactions due to <u>linearly varying distributed load</u> (LDL)**

In [8]:
def reactions_LDL(n): 
    xStart = linearLoads[n,0]   #Location of start of LDL
    xEnd = linearLoads[n,1]     #Location of end of LDL
    fy_start = linearLoads[n,2] #Magnitude of LDL on left side
    fy_end = linearLoads[n,3]   #Magnitude of LDL on right side
    
    #Determine location and magnitude of force resultant
    if abs(fy_start)>0:
        fy_Res = 0.5*fy_start*(xEnd-xStart)  #Magnitude of resultant 
        x_Res = xStart + (1/3)*(xEnd-xStart) #Location of resultant
    else:
        fy_Res = 0.5*fy_end*(xEnd-xStart) 
        x_Res = xStart + (2/3)*(xEnd-xStart)
        
    
    #SAME AS UDL FUNCTION FROM HERE 
    la_p = A-x_Res   #Lever arm of resultant load about point A
    mp = fy_Res*la_p #Moment generated by resultant load about A - closkwise moments are positive    
    la_vb = B-A      #Lever arm of vertical reaction at B about point A

    Vb = mp/la_vb   #Vertical reaction at B
    Va = -fy_Res-Vb #Vertical reaction at A    
    
    return Va, Vb

**2.5 Cycle through all <u>point loads</u> and determine reactions**

In [9]:
PL_record = np.empty([0,3])
if(nPL>0):
    for n, p in enumerate(pointLoads):
        va, vb, ha = reactions_PL(n) #Calculate reactions
        PL_record = np.append(PL_record, [np.array([va, ha, vb])], axis=0) #Store reactions for each point load

        #Add reactions to record (superposition)
        reactions[0] = reactions[0] + va 
        reactions[1] = reactions[1] + ha 
        reactions[2] = reactions[2] + vb 

**2.6 Cycle through all <u>point moments</u> and determine reactions**

In [10]:
PM_record = np.empty([0,2])
if(nPM>0):
    for n, p in enumerate(pointMoments):
        va, vb = reactions_PM(n) #Calculate reactions
        PM_record = np.append(PM_record, [np.array([va, vb])], axis=0) #Store reactions for each point moment

        #Add reactions to record (superposition)
        reactions[0] = reactions[0] + va 
        reactions[2] = reactions[2] + vb         

**2.7 Cycle through all <u>UDLs</u> and determine reactions**

In [11]:
UDL_record = np.empty([0,2])
if(nUDL>0):
    for n, p in enumerate(distributedLoads):
        va, vb = reactions_UDL(n) #Calculate reactions
        UDL_record = np.append(UDL_record, [np.array([va, vb])], axis=0) #Store reactions for each distributed load

        #Add reactions to record (superposition)
        reactions[0] = reactions[0] + va 
        reactions[2] = reactions[2] + vb 

**2.8 Cycle through all <u>LDLs</u> and determine reactions**

In [12]:
LDL_record = np.empty([0,2])
if(nLDL>0):
    for n, p in enumerate(linearLoads):
        va, vb = reactions_LDL(n) #Calculate reactions
        LDL_record = np.append(LDL_record, [np.array([va, vb])], axis=0) #Store reactions for each linearly distributed load

        #Add reactions to record (superposition)
        reactions[0] = reactions[0] + va 
        reactions[2] = reactions[2] + vb 

<hr>

## 3.0 Shear and Moment Calculation

**3.1 Define function to calculate shear forces and bending moments due to <u>point loads</u>**

In [13]:
def shear_moment_PL(n):    
    xp = pointLoads[n,0] #Location of point load
    fy = pointLoads[n,2] #Point load vertical component magnitude 
    Va = PL_record[n,0]  #Vertical reaction at A for this point load
    Vb = PL_record[n,2]  #Vertical reaction at B for this point load
    
    #Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this point load
    Moment = np.zeros(len(X)) #Initialise a container to hold all moment force data for this point load
    for i, x in enumerate(X):    
        shear = 0  #Initialise the shear force for this data point
        moment = 0 #Initialise the bending moment for this data point

        if x>A:
            #Calculate shear and moment from reaction at A
            shear = shear + Va
            moment = moment - Va*(x-A)

        if x>B:
            #Calculate shear and moment from reaction at B
            shear = shear + Vb
            moment = moment - Vb*(x-B)

        if x>xp:
            #Calculate shear and moment from point load
            shear = shear + fy
            moment = moment - fy*(x-xp)

        #Store shear and moment for this location
        Shear[i] = shear
        Moment[i] = moment

    return Shear, Moment

**3.2 Define function to calculate shear forces and bending moments due to <u>point moments</u>**

In [14]:
def shear_moment_PM(n):    
    xm = pointMoments[n,0] #Location of point moment
    m = pointMoments[n,1]  #Point moment magnitude 
    Va = PM_record[n,0]    #Vertical reaction at A for this point moment
    Vb = PM_record[n,1]    #Vertical reaction at B for this point moment
    
    #Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this point moment
    Moment = np.zeros(len(X)) #Initialise a container to hold all moment force data for this point moment
    for i, x in enumerate(X):    
        shear = 0  #Initialise the shear force for this data point
        moment = 0 #Initialise the bending moment for this data point

        if x>A:
            #Calculate shear and moment from reaction at A
            shear = shear + Va
            moment = moment - Va*(x-A)

        if x>B:
            #Calculate shear and moment from reaction at B
            shear = shear + Vb
            moment = moment - Vb*(x-B)

        if x>xm:
            #Calculate moment due to point moment (No influence on shear)            
            moment = moment - m

        #Store shear and moment for this location
        Shear[i] = shear
        Moment[i] = moment

    return Shear, Moment

**3.3 Define function to calculate shear forces and bending moments due to <u>UDLs</u>**

In [15]:
def shear_moment_UDL(n):    
    xStart = distributedLoads[n,0] #Location of start of UDL
    xEnd = distributedLoads[n,1]   #Location of end of UDL
    fy = distributedLoads[n,2]     #Magnitude of UDL
    Va = UDL_record[n,0] #Vertical reaction at A for this UDL
    Vb = UDL_record[n,1] #Vertical reaction at B for this UDL
    
    #Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this UDL
    Moment = np.zeros(len(X)) #Initialise a container to hold all moment force data for this UDL
    for i, x in enumerate(X):    
        shear = 0  #Initialise the shear force for this data point
        moment = 0 #Initialise the bending moment for this data point
                
        if x>A:
            #Calculate shear and moment from reaction at A
            shear = shear + Va
            moment = moment - Va*(x-A)

        if x>B:
            #Calculate shear and moment from reaction at B
            shear = shear + Vb
            moment = moment - Vb*(x-B)
        
        if x>xStart and x<=xEnd:
            #Cut through UDL - calculate shear and moment from UDL
            shear = shear + fy*(x-xStart)
            moment = moment - fy*(x-xStart)*0.5*(x-xStart)
        elif x>xEnd:
            #Cut to right of UDL - calculate shear and moment from UDL
            shear = shear + fy*(xEnd-xStart)
            moment = moment - fy*(xEnd-xStart)*(x - xStart - 0.5*(xEnd-xStart))

        #Store shear and moment for this location
        Shear[i] = shear
        Moment[i] = moment    

    return Shear, Moment                

**3.4 Define function to calculate shear forces and bending moments due to <u>LDLs</u>**

In [16]:
def shear_moment_LDL(n): 
    xStart = linearLoads[n,0]   #Location of start of LDL
    xEnd = linearLoads[n,1]     #Location of end of LDL
    fy_start = linearLoads[n,2] #Magnitude of left side of LDL
    fy_end = linearLoads[n,3]   #Magnitude of right side of LDL
    Va = LDL_record[n,0] #Vertical reaction at A for this LDL
    Vb = LDL_record[n,1] #Vertical reaction at B for this LDL
    
    #Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this LDL
    Moment = np.zeros(len(X)) #Initialise a container to hold all moment force data for this LDL
    for i, x in enumerate(X):    
        shear = 0  #Initialise the shear force for this data point
        moment = 0 #Initialise the bending moment for this data point
                
        if x>A:
            #Calculate shear and moment from reaction at A
            shear = shear + Va
            moment = moment - Va*(x-A)

        if x>B:
            #Calculate shear and moment from reaction at B
            shear = shear + Vb
            moment = moment - Vb*(x-B)
        
        if x>xStart and x<=xEnd:
            #Cut through LDL - calculate shear and moment from LDL
            if abs(fy_start)>0:
                x_base = x-xStart #Base of the triangular load distribution
                f_cut = fy_start - x_base*(fy_start/(xEnd-xStart)) #Magnitude of LDL at cut
                R1 = 0.5*x_base*(fy_start-f_cut) #Magnitude of resultant for triangular portion of load
                R2 = x_base*f_cut #Magnitude of resultant for constant portion of load 
                shear = shear + R1 + R2
                moment = moment - R1*(2/3)*x_base - R2*(x_base/2)
            else:
                x_base = x-xStart #Base of the triangular load distribution
                f_cut = fy_end*(x_base/(xEnd-xStart)) #Magnitude of LDL at cut
                R = 0.5*x_base*f_cut #Magnitude of resultant
                shear = shear + R
                moment = moment - R*(x_base/3)
                
        elif x>xEnd:
            #Cut to right of LDL - calculate shear and moment from LDL
            if abs(fy_start)>0:
                R = 0.5*fy_start*(xEnd-xStart)
                xr = xStart + (1/3)*(xEnd-xStart)                
                shear = shear + R
                moment = moment - R*(x-xr)
            else:
                R = 0.5*fy_end*(xEnd-xStart)
                xr = xStart + (2/3)*(xEnd-xStart)                
                shear = shear + R
                moment = moment - R*(x-xr)
                

        #Store shear and moment effects for this location
        Shear[i] = shear
        Moment[i] = moment    

    return Shear, Moment                

**3.5 Cycle through all <u>point loads</u> and determine shear and moment**

In [17]:
#Calculate the shear force and bending moment at each datapoint due to point load
if(nPL>0):
    for n, p in enumerate(pointLoads):
        Shear, Moment = shear_moment_PL(n)
        shearForce = np.append(shearForce, [Shear], axis=0)        #Store shear force record for each point load
        bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each point load

**3.6 Cycle through all <u>point moments</u> and determine moment**

In [18]:
#Calculate the shear force and bending moment at each datapoint due to point moment
if(nPM>0):
    for n, p in enumerate(pointMoments):
        Shear, Moment = shear_moment_PM(n)
        shearForce = np.append(shearForce, [Shear], axis=0)        #Store shear force record for each point moment
        bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each point moment

**3.7 Cycle through all <u>UDLs</u> and determine shear and moment**

In [19]:
#Calculate the shear force and bending moment at each datapoint due to UDL
if(nUDL>0):
    for n, p in enumerate(distributedLoads):
        Shear, Moment = shear_moment_UDL(n)
        shearForce = np.append(shearForce, [Shear], axis=0)        #Store shear force record for each UDL
        bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each UDL

**3.8 Cycle through all <u>LDLs</u> and determine shear and moment**

In [20]:
#Calculate the shear force and bending moment at each datapoint due to LDL
if(nLDL>0):
    for n, p in enumerate(linearLoads):
        Shear, Moment = shear_moment_LDL(n)
        shearForce = np.append(shearForce, [Shear], axis=0) #Store shear force record for each LDL
        bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each LDL

<hr>

## 4.0 Plotting and Printing

In [21]:
print('The vertical reaction at A is {one} kN'.format(one=round(reactions[0],2)))
print('The vertical reaction at B is {one} kN'.format(one=round(reactions[2],2)))
print('The horizontal reaction at A is {one} kN'.format(one=round(reactions[1],2)))

The vertical reaction at A is 1.4 kN
The vertical reaction at B is 1.4 kN
The horizontal reaction at A is 0.0 kN


**Plotly**
<a href="https://plotly.com/python/">🔗  Plotly reference docs</a>

In [22]:
import plotly as py #Import Plotly 
import plotly.graph_objs as go #Import graph objects
py.offline.init_notebook_mode(connected=True) #Setup offline plotting (inside Jupyter notebook)

In [23]:
#Define the layout object
layout = go.Layout(
    title={
        'text': "Shear Force Diagram",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    titlefont=dict(size=15),
    yaxis = dict(
        title='Shear Force (kN)'
    ),
    xaxis = dict(
        title='Distance (m)',       
        range=[-1, span+1]
    ),
    showlegend=False,        
)

#Define the shear force trace
line = go.Scatter(
    x = X,
    y = sum(shearForce),
    mode='lines',
    name='Shear Force',
    fill='tonexty',
    line_color='green',
    fillcolor='rgba(0, 255, 0, 0.1)'
)

#Define a horizontal line to represent the structure
axis = go.Scatter(
    x = [0, span],
    y = [0,0],
    mode='lines',
    line_color='black'
)

#Generate and view the figure
fig = go.Figure(data=[line, axis], layout=layout)

py.offline.iplot(fig)


In [24]:
#Define the layout object
layout = go.Layout(
    title={
        'text': "Bending Moment Diagram",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    titlefont=dict(size=15),
    yaxis = dict(
        title='Bending Moment (kNm)',
        autorange="reversed",   
    ),
    xaxis = dict(
        title='Distance (m)',       
        range=[-1, span+1]
    ),
    showlegend=False 
)

#Define the Bending Moment trace
line = go.Scatter(
    x = X,
    y = -sum(bendingMoment),
    mode='lines',
    name='Bending Moment',
    fill='tonexty',
    line_color='red',
    fillcolor='rgba(255, 0, 0, 0.1)'
)

#Define a horizontal line to represent the structure
axis = go.Scatter(
    x = [0, span],
    y = [0,0],
    mode='lines',
    line_color='black'
)

#Generate and view the figure
fig = go.Figure(data=[line, axis], layout=layout)
py.offline.iplot(fig)


<hr>

## 5.0 Deflection Calculation

Numerical solution of the Differential Equation of the Deflection Curve

# **5.1 User input**

In [25]:
#INPUT GUESS FOR INITIAL ROTATION AT SUPPORT A AND SWEEP STEP 
deltaRot = 0.000005 #The step size in rotation guesses
initRot = -0.0156 #-856 Initial value for rotation at support A (ASSUMED VALUE)
E = 200*10**6 #(N/m^2) Young's modulus
I = 3.63427e-7 #(m^4) Second moment of area

**5.2 Constants and initialisations**

In [26]:
#Initial constants
M = -sum(bendingMoment) #Bending moment values used for integration
delX = X[1]-X[0] #Distance between bending moment values
EI=E*I
initDef = 0 #Initial value for displacement at support A
supportIndexA = np.where(X == A)[0].item() #The index of values at support A
supportIndexB = np.where(X == B)[0].item() #The index of values at support B

**5.3 Define a function to calculate deflection by forward integrating (from left support) using the trapezoidal rule**

In [27]:
def calcDeflection(M, EI, delX, theta_0, v_0):            
    #Re-assign input arguments to variables used in function
    theta_im1 = theta_0 #Initial rotation
    v_im1 = v_0 #Initial displacement

    #Initialise containers to hold Rotation and Deflection
    Rotation = np.zeros(len(X))
    Rotation[supportIndexA] = theta_im1
    Deflection = np.zeros(len(X))
    Deflection[supportIndexA] = v_im1

    #Loop through data and integrate (Trapezoidal rule)
    for i, m in enumerate(M[supportIndexA::]):
        ind = i + supportIndexA #Take account of fact that support A might not be at start of beam
        if i>0:        
            M_im1 = M[ind-1]
            M_i = M[ind]        
            M_avg = 0.5*(M_i + M_im1)        

            theta_i = theta_im1 + (M_avg/EI)*delX #Integrate moment values to get rotations                     
            v_i = v_im1 + 0.5*(theta_i+theta_im1)*delX #Integrate rotation values to get displacements

            #Store data
            Rotation[ind] = theta_i        
            Deflection[ind] = v_i

            #Update values for next loop iteration
            theta_im1 = theta_i     
            v_im1 = v_i
            
    return Rotation, Deflection        

**5.4 Define a function to sweep the range of initial rotation values to identify the one that minimised deflection at right support**

In [28]:
def zeroCrossing(Deflection, guessStep, initRot, initDef):
    """
    Find the value of initial rotation that minimised deflection at right side 
    support by identifying where error crosses zero.
    """
    
    #If the deflection error is positive
    if Deflection[supportIndexB]>0:
        errorIsPositive = True #Set flag for error sign
        
        #Keep testing lower initial rotation values until error turns NEGATIVE
        while errorIsPositive:
            initRot = initRot + guessStep
            Rotation, Deflection = calcDeflection(M, EI, delX, initRot, initDef)
            
            #If error has turned NEGATIVE, switch the flag to allow loop to stop
            if Deflection[supportIndexB]<0:
                errorIsPositive = False
                solvedInitRotation = initRot #Save the 'solved' value that minimised the error
    
    #Else if deflection error is negative
    elif Deflection[supportIndexB]<0:
        errorIsPositive = False #Set flag for error sign
        
        #Keep testing lower initial rotation values until error turns POSITIVE
        while not errorIsPositive:
            initRot = initRot + guessStep
            Rotation, Deflection = calcDeflection(M, EI, delX, initRot, initDef)
            
            #If error has turned POSITIVE, switch the flag to allow loop to stop
            if Deflection[supportIndexB]>0:
                errorIsPositive = True
                solvedInitRotation = initRot #Save the 'solved' value that minimised the error
    
    return solvedInitRotation    

**5.5 Test whether we need to sweep for initial rotation values in the positive or negative direction and call sweeper function from above**

In [29]:
#Test whether reducing or increasing initial rotation leads to reduction in disp error at other support
testDef = np.zeros(3)
for i, r in enumerate([initRot-deltaRot, initRot, initRot+deltaRot]):
    Rotation, Deflection = calcDeflection(M, EI, delX, r, initDef)
    testDef[i] = Deflection[supportIndexB]
        
if(abs(testDef[0])<abs(testDef[1])):
    #Need to test in the negative rotation direction by reducing the initial rotation guess
    print('Need to test in the negative direction')    
    solvedInitRotation = zeroCrossing(Deflection, -deltaRot, initRot, initDef)            

elif(abs(testDef[2])<abs(testDef[1])):
    #Need to test in the positive rotation direction by incresing the initial rotation guess 
    print('Need to test in the positive direction')    
    solvedInitRotation = zeroCrossing(Deflection, deltaRot, initRot, initDef)      

#Run the deflection calculation with the solved value of initial rotation
Rotation, Deflection = calcDeflection(M, EI, delX, solvedInitRotation, initDef)    
print('Solved initial rotation is {one}'.format(one=solvedInitRotation))
print('The error in displacement at support is {one}'.format(one=Deflection[supportIndexB]))
    

Need to test in the negative direction
Solved initial rotation is -0.015650000000000004
The error in displacement at support is -1.094167466850649e-06


**5.6 Solve for displacement to left of support A (if required)**

In [30]:
if A!=0:
    print("There is an overhand on the left side - solve for deflection by integrating in reverse direction")
    
    theta_im1 = -solvedInitRotation #Rotation on other side of support A
    v_im1 = 0 #Vertical deflection at support A

    #Generate a range of indices in reverse direction from support A to left endge of beam
    reverseRange = np.arange(supportIndexA-1,-1,-1) 

    #Loop through data and integrate (Trapezoidal rule) - REVERSE DIRECTION
    for i in reverseRange:                        
        M_im1 = M[i+1] #(300) - Assign previous value of M (reverse direction)
        M_i = M[i] #(299) - Assign current value of M (reverse direction)
        M_avg = 0.5*(M_i + M_im1)   
        
        theta_i = theta_im1 + (M_avg/EI)*delX #Integrate moment values to get rotations                     
        v_i = v_im1 + 0.5*(theta_i+theta_im1)*delX #Integrate rotation values to get displacements
        
        #Store data
        Rotation[i] = theta_i        
        Deflection[i] = v_i

        #Update values for next loop iteration
        theta_im1 = theta_i     
        v_im1 = v_i                        

**5.7 Plotting deflection**

In [31]:
#------------------ PRINTING -----------------------------
#Define the layout object
layout = go.Layout(
    title={
        'text': "Deflection",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    titlefont=dict(size=15),
    yaxis = dict(
        title='Deflection'
    ),
    xaxis = dict(
        title='Distance (m)',       
        range=[-1, span+1]
    ),
    showlegend=False,        
)

#Define the shear force trace
line = go.Scatter(
    x = X,
    y = Deflection,
    mode='lines',
    name='Deflection',
    line_color='orange',
    fill='tonexty',
    fillcolor='rgba(255, 255, 0, 0.1)'
)

#Define a horizontal line to represent the structure
axis = go.Scatter(
    x = [0, span],
    y = [0,0],
    mode='lines',
    line_color='black'
)

#Generate and view the figure
fig = go.Figure(data=[line,axis], layout=layout)

py.offline.iplot(fig)

## 6.0 Deflection Validation

**Analytical solution using Macauley's method - EXAMPLE WITH OVERHANGS**

<a href="https://www.degreetutors.com/macauleys-method-beam-deflection/">Macauley's Method Example</a> on DegreeTutors.

In [32]:
#Cycle through the structure and calculate the deflection at each point
Deflection_Test = np.zeros(len(X))  #Initialise a container to hold all deflections
for i, x in enumerate(X):    
    deflection = 0

    #Term 1 and 7
    if x>0:
        #Moment
        delta = -(60/2)*x**2
        deflection = deflection + delta
        
        #C1
        delta = -765*x
        deflection = deflection + delta
        
    #Term 2 - Reaction at A
    if x-3 > 0:
        delta = (123.5/6)*(x-3)**3
        deflection = deflection + delta

    #Term 3 - 50 kN/m UDL acting downwards
    if x-5 > 0:
        delta = -(50/24)*(x-5)**4
        deflection = deflection + delta
    
    #Term 4 - 100kN point load
    if x-11 > 0:    
        delta = -(100/6)*(x-11)**3
        deflection = deflection + delta
        
    #Term 5 - 50 kN/m UDL acting upwards
    if x-9 > 0:
        delta = (50/24)*(x-9)**4
        deflection = deflection + delta
        
    #Term 6 - Reaction at B
    if x-13 > 0:
        delta = (251.5/6)*(x-13)**3
        deflection = deflection + delta
        
    #Term 7 - C2
    deflection = deflection + 2565               

    #Store deflection for this location
    Deflection_Test[i] = deflection/EI  

In [33]:
#------------------ PRINTING -----------------------------
#Define the layout object
layout = go.Layout(
    title={
        'text': "Deflection",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    titlefont=dict(size=15),
    yaxis = dict(
        title='Deflection'
    ),
    xaxis = dict(
        title='Distance (m)',       
        range=[-1, span+1]
    ),
    showlegend=True,        
)

#Define the shear force trace
line_Macauley = go.Scatter(
    x = X,
    y = Deflection_Test,
    mode='lines',
    name='Macauley',
    line_color='green',
#     fill='tonexty',
#     fillcolor='rgba(255, 255, 0, 0.1)'
)

#Define the shear force trace
line_Numerical = go.Scatter(
    x = X,
    y = Deflection,
    mode='lines',
    name='Numerical',
    line_color='orange',
#     fill='tonexty',
#     fillcolor='rgba(255, 255, 0, 0.1)'
)

#Define a horizontal line to represent the structure
axis = go.Scatter(
    x = [0, span],
    y = [0,0],
    mode='lines',
    line_color='black'
)

#Generate and view the figure
fig = go.Figure(data=[line_Macauley, line_Numerical,axis], layout=layout)

py.offline.iplot(fig)